# Analyse the time measurements of Metavision

This notebook can be used for a time measurement analysis on data from a pilot or for post market surveillance (PMS).


## Load data

### Load packages

In [ ]:
from datetime import date, timedelta
from enum import Enum
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import mannwhitneyu


### Specify time period
Adjust the MODE to indicate whether this notebook is analyzing PMS or PILOT time periods.

In [ ]:
class MeasureMode(Enum):
    PILOT = "Pilot"
    PMS = "PMS"


MODE = MeasureMode.PMS

BASELINE_START_DATE = date(2024, 9, 18)
BASELINE_END_DATE = date(2024, 10, 15)
START_DATE = date(2025, 4, 1)
END_DATE = date(2025, 7, 1)

if MODE == MeasureMode.PILOT:
    MID_DATE = date(2025, 5, 15)
    OFFLINE_START_DATE = date(2025, 8, 1)
elif MODE == MeasureMode.PMS:
    MID_DATE = None
    OFFLINE_START_DATE = None

NAME = MODE.value

print(f"{NAME} dates are used for further analysis in this notebook.")


### Analysis datadump
The file "metavision_time_measurements.csv" can be updated by running 'export_time_measurements.sql' under the data folder. 

In [ ]:
df_total = pd.read_csv(
    Path.cwd().parent / "data" / "raw" / "metavision_time_measurements.csv",
    sep=",",
    parse_dates=[
        "AddmissionDate",
        "DischargeDate",
        "FormRelease",
        "SessieCreate",
        "StartSchrijven",
        "EindeSchrijven",
    ],
)
df_total.rename(columns={"AddmissionDate": "AdmissionDate"}, inplace=True)
df_total

In [ ]:
df_total = df_total[df_total["Schrijven_minuten"] >= 0]
df_total = df_total[df_total["Schrijven_minuten"] < 180]  # Remove outliers
df_total["Schrijven_minuten"].hist(bins=100)


## Filtering

### Explanation of the filtering

The baseline should contain all admissions where no AI was used and time measurement is available. Time measurements are available when a patient was admitted after the BASELINE_START_DATE. AI was not used when either the discharge date is before the BASELINE_END_DATE or when they remained in a session created before the BASELINE_END_DATE. It is important to note that, in case of a pilot, the AI was offline after OFFLINE_START_DATE. However, for patients in the baseline group as described above, they were not affected by this as they were not using AI so no additional filtering is needed. There are two options for the baseline group:
- Baseline: the admission date falls in between BASELINE_START_DATE and BASELINE_END_DATE
- Baseline strictly: the admission and discharge date fall in between BASELINE_START_DATE and BASELINE_END_DATE
- Baseline (strict) with admissions less than one week
- Baseline (strict) with admissions more than one week

The AI measurement should contain all admissions where AI was used (and time measurement is available but there is no option for AI without time measurement). AI was used when the admission date is after the START_DATE. 
- AI during the pilot/ pms timeperiod: the admission date falls in between START_DATE and END_DATE
- AI strictly during the pilot/ pms timeperiod: the admission and discharge date fall in between START_DATE and END_DATE
- AI during the pilot (strict) with admissions less than one week
- AI during the pilot (strict) with admissions more than one week

Pilot specific information: as the AI was also utilized after the pilot ended there are a few different analyses we will perform. It is important to note that the AI was offline after OFFLINE_START_DATE, so we will not consider any admissions where the discharge date is after this date. Additional analysis for the pilot (which are not executed for the pms analysis) are:
- AI during first half of the pilot: the admission date is in between the START_DATE and the MID_DATE, regardless of the discharge date
- AI strictly during first half of the pilot: the admission and discharge date fall in between the PILOT_START_DATE and the PILOT_MID_DATE
- AI during second half of the pilot: the admission date is in between the MID_DATE and the END_DATE, regardless of the discharge date
- AI strictly during second half of the pilot: the admission and discharge date fall in between the MID_DATE and the END_DATE
- AI strictly after the pilot: the admission date and discharge date fall in between the END_DATE and the OFFLINE_START_DATE
- AI during the "extended" pilot: the admission date is in between the START_DATE and the OFFLINE_START_DATE. This is similar to the "strict" setup above.

### Filtering for baseline data

In [ ]:
baseline_df = df_total.copy()
baseline_strict_df = df_total.copy()
baseline_short_df = df_total.copy()
baseline_long_df = df_total.copy()

baseline_strict_no_short_df = df_total.copy()

baseline_df = baseline_df[
    (baseline_df["AdmissionDate"].dt.date >= BASELINE_START_DATE)
    & (
        (baseline_df["DischargeDate"].dt.date < BASELINE_END_DATE)
        | baseline_df.groupby("PatientID")["SessieCreate"].transform(
            lambda x: (x.dt.date < BASELINE_END_DATE).all()
        )
    )
]

baseline_strict_df = baseline_strict_df[
    (baseline_strict_df["AdmissionDate"].dt.date >= BASELINE_START_DATE)
    & (baseline_strict_df["DischargeDate"].dt.date < BASELINE_END_DATE)
]

baseline_short_df = baseline_short_df[
    (baseline_short_df["AdmissionDate"].dt.date >= BASELINE_START_DATE)
    & (baseline_short_df["DischargeDate"].dt.date < BASELINE_END_DATE)
    & (
        (
            baseline_short_df["DischargeDate"].dt.date
            - baseline_short_df["AdmissionDate"].dt.date
        )
        < timedelta(days=7)
    )
]

baseline_long_df = baseline_long_df[
    (baseline_long_df["AdmissionDate"].dt.date >= BASELINE_START_DATE)
    & (baseline_long_df["DischargeDate"].dt.date < BASELINE_END_DATE)
    & (
        (
            baseline_long_df["DischargeDate"].dt.date
            - baseline_long_df["AdmissionDate"].dt.date
        )
        >= timedelta(days=7)
    )
]

baseline_strict_no_short_df = baseline_strict_no_short_df[
    (baseline_strict_no_short_df["AdmissionDate"].dt.date >= BASELINE_START_DATE)
    & (baseline_strict_no_short_df["DischargeDate"].dt.date < BASELINE_END_DATE)
    & (
        (
            baseline_strict_no_short_df["DischargeDate"].dt.date
            - baseline_strict_no_short_df["AdmissionDate"].dt.date
        )
        > timedelta(days=2)
    )
]

### Filtering for Pilot or PMS data

In [ ]:
ai_df = df_total.copy()
ai_strict_df = df_total.copy()
ai_short_df = df_total.copy()
ai_long_df = df_total.copy()

ai_no_short_df = df_total.copy()

ai_df = ai_df[
    (ai_df["AdmissionDate"].dt.date >= START_DATE)
    & (ai_df["AdmissionDate"].dt.date < END_DATE)
]

ai_strict_df = ai_strict_df[
    (ai_strict_df["AdmissionDate"].dt.date >= START_DATE)
    & (ai_strict_df["DischargeDate"].dt.date < END_DATE)
]

ai_short_df = ai_short_df[
    (ai_short_df["AdmissionDate"].dt.date >= START_DATE)
    & (ai_short_df["DischargeDate"].dt.date < END_DATE)
    & (
        (ai_short_df["DischargeDate"].dt.date - ai_short_df["AdmissionDate"].dt.date)
        < timedelta(days=7)
    )
]

ai_long_df = ai_long_df[
    (ai_long_df["AdmissionDate"].dt.date >= START_DATE)
    & (ai_long_df["DischargeDate"].dt.date < END_DATE)
    & (
        (ai_long_df["DischargeDate"].dt.date - ai_long_df["AdmissionDate"].dt.date)
        >= timedelta(days=7)
    )
]

ai_no_short_df = ai_no_short_df[
    (ai_no_short_df["AdmissionDate"].dt.date >= START_DATE)
    & (ai_no_short_df["DischargeDate"].dt.date < END_DATE)
    & (
        (
            ai_no_short_df["DischargeDate"].dt.date
            - ai_no_short_df["AdmissionDate"].dt.date
        )
        > timedelta(days=2)
    )
]

### Additional filtering for pilot data

In [ ]:
ai_first_half_df = df_total.copy()
ai_first_half_strict_df = df_total.copy()
ai_second_half_df = df_total.copy()
ai_second_half_strict_df = df_total.copy()

ai_first_half_df = ai_first_half_df[
    (ai_first_half_df["AdmissionDate"].dt.date >= START_DATE)
    & (ai_first_half_df["AdmissionDate"].dt.date < MID_DATE)
]

ai_first_half_strict_df = ai_first_half_strict_df[
    (ai_first_half_strict_df["AdmissionDate"].dt.date >= START_DATE)
    & (ai_first_half_strict_df["DischargeDate"].dt.date < MID_DATE)
]

ai_second_half_df = ai_second_half_df[
    (ai_second_half_df["AdmissionDate"].dt.date >= MID_DATE)
    & (ai_second_half_df["AdmissionDate"].dt.date < END_DATE)
]

ai_second_half_strict_df = ai_second_half_strict_df[
    (ai_second_half_strict_df["AdmissionDate"].dt.date >= MID_DATE)
    & (ai_second_half_strict_df["DischargeDate"].dt.date < END_DATE)
]

# optional additional analysis if pilot end date is extended to offline start date
ai_post_strict_df = df_total.copy()
ai_extended_df = df_total.copy()

ai_post_strict_df = ai_post_strict_df[
    (ai_post_strict_df["AdmissionDate"].dt.date >= END_DATE)
    & (ai_post_strict_df["DischargeDate"].dt.date < OFFLINE_START_DATE)
]

ai_extended_df = ai_extended_df[
    (ai_extended_df["AdmissionDate"].dt.date >= START_DATE)
    & (ai_extended_df["DischargeDate"].dt.date < OFFLINE_START_DATE)
]


## Result analysis

### Outcomes in numbers

In [ ]:
print("Baseline measurement outcomes:")

print(
    f"Number of patients in baseline measurement: {baseline_df['PatientID'].nunique()}"
)
print(
    f"Number of patients in baseline measurement (strict): "
    f"{baseline_strict_df['PatientID'].nunique()}"
)
print(
    f"Number of patients in baseline measurement (strict & short): "
    f"{baseline_short_df['PatientID'].nunique()}"
)
print(
    f"Number of patients in baseline measurement (strict & long): "
    f"{baseline_long_df['PatientID'].nunique()}"
)
print(
    f"Number of patients in baseline measurement (strict & no short): "
    f"{baseline_strict_no_short_df['PatientID'].nunique()}"
)
print("\n")

print(f"{NAME} measurement outcomes:")
print(
    f"Number of patients in AI measurement during the {NAME}: "
    f"{ai_df['PatientID'].nunique()}"
)
print(
    f"Number of patients in AI measurement during the {NAME} (strict): "
    f"{ai_strict_df['PatientID'].nunique()}"
)
print(
    f"Number of patients in AI measurement during the {NAME} (strict & short): "
    f"{ai_short_df['PatientID'].nunique()}"
)
print(
    f"Number of patients in AI measurement during the {NAME} (strict & long): "
    f"{ai_long_df['PatientID'].nunique()}"
)
print(
    f"Number of patients in AI measurement during the {NAME} (no short): "
    f"{ai_no_short_df['PatientID'].nunique()}"
)
print("\n")


if MODE != MeasureMode.PMS and MODE == MeasureMode.PILOT:
    print("Pilot specific measurement outcomes:")
    print(
        f"Number of patients in AI measurement during the first half of the pilot: "
        f"{ai_first_half_df['PatientID'].nunique()}"
    )

    print(
        f"Number of patients in AI measurement during the first half of the pilot "
        f"(strict): "
        f"{ai_first_half_strict_df['PatientID'].nunique()}"
    )

    print(
        f"Number of patients in AI measurement during the second half of the pilot: "
        f"{ai_second_half_df['PatientID'].nunique()}"
    )
    print(
        "Number of patients in AI measurement during the second half of the pilot "
        "(strict): "
        f"{ai_second_half_strict_df['PatientID'].nunique()}"
    )
    print(
        f"Number of patients in AI measurement post pilot (strict): "
        f"{ai_post_strict_df['PatientID'].nunique()}"
    )
    print(
        f"Number of patients in AI measurement during the extended pilot: "
        f"{ai_extended_df['PatientID'].nunique()}"
    )

### Differences in average time measurements between baseline and pilot/ PMS

In [ ]:
baseline_sum = baseline_df.groupby("PatientID")["Schrijven_minuten"].sum()
baseline_strict_sum = baseline_strict_df.groupby("PatientID")["Schrijven_minuten"].sum()
baseline_short_sum = baseline_short_df.groupby("PatientID")["Schrijven_minuten"].sum()
baseline_long_sum = baseline_long_df.groupby("PatientID")["Schrijven_minuten"].sum()
baseline_strict_no_short_sum = baseline_strict_no_short_df.groupby("PatientID")[
    "Schrijven_minuten"
].sum()

ai_sum = ai_df.groupby("PatientID")["Schrijven_minuten"].sum()
ai_strict_sum = ai_strict_df.groupby("PatientID")["Schrijven_minuten"].sum()
ai_short_sum = ai_short_df.groupby("PatientID")["Schrijven_minuten"].sum()
ai_long_sum = ai_long_df.groupby("PatientID")["Schrijven_minuten"].sum()
ai_no_short_sum = ai_no_short_df.groupby("PatientID")["Schrijven_minuten"].sum()

print(f"Average time baseline: {baseline_sum.mean():.2f} ({baseline_sum.std():.2f})")
print(f"Average time during {NAME}: {ai_sum.mean():.2f} ({ai_sum.std():.2f})")

print(
    f"Average time baseline (strict): {baseline_strict_sum.mean():.2f} "
    f"({baseline_strict_sum.std():.2f})"
)
print(
    f"Average time baseline (strict & short): {baseline_short_sum.mean():.2f} "
    f"({baseline_short_sum.std():.2f})"
)
print(
    f"Average time baseline (strict & long): {baseline_long_sum.mean():.2f} "
    f"({baseline_long_sum.std():.2f})"
)
print(
    f"Average time baseline (strict & no short): "
    f"{baseline_strict_no_short_sum.mean():.2f} "
    f"({baseline_strict_no_short_sum.std():.2f})"
)

print(
    f"Average time during {NAME} (strict): {ai_strict_sum.mean():.2f} "
    f"({ai_strict_sum.std():.2f})"
)
print(
    f"Average time during {NAME} (strict & short): "
    f"{ai_short_sum.mean():.2f} ({ai_short_sum.std():.2f})"
)
print(
    f"Average time during {NAME} (strict & long): "
    f"{ai_long_sum.mean():.2f} ({ai_long_sum.std():.2f})"
)
print(
    f"Average time during {NAME} (no short): {ai_no_short_sum.mean():.2f} "
    f"({ai_no_short_sum.std():.2f})"
)

if MODE != MeasureMode.PMS and MODE == MeasureMode.PILOT:
    ai_first_half_sum = ai_first_half_df.groupby("PatientID")["Schrijven_minuten"].sum()
    ai_first_half_strict_sum = ai_first_half_strict_df.groupby("PatientID")[
        "Schrijven_minuten"
    ].sum()
    ai_second_half_sum = ai_second_half_df.groupby("PatientID")[
        "Schrijven_minuten"
    ].sum()
    ai_second_half_strict_sum = ai_second_half_strict_df.groupby("PatientID")[
        "Schrijven_minuten"
    ].sum()
    ai_post_strict_sum = ai_post_strict_df.groupby("PatientID")[
        "Schrijven_minuten"
    ].sum()
    ai_extended_sum = ai_extended_df.groupby("PatientID")["Schrijven_minuten"].sum()

    print(
        f"Average time during the first half of the pilot: "
        f"{ai_first_half_sum.mean():.2f} ({ai_first_half_sum.std():.2f})"
    )
    print(
        f"Average time during the second half of the pilot: "
        f"{ai_second_half_sum.mean():.2f} ({ai_second_half_sum.std():.2f})"
    )

    print(
        f"Average time during the first half of the pilot (strict): "
        f"{ai_first_half_strict_sum.mean():.2f} "
        f"({ai_first_half_strict_sum.std():.2f})"
    )
    print(
        f"Average time during the second half of the pilot (strict): "
        f"{ai_second_half_strict_sum.mean():.2f} "
        f"({ai_second_half_strict_sum.std():.2f})"
    )
    print(
        f"Average time post pilot (strict): {ai_post_strict_sum.mean():.2f} "
        f"({ai_post_strict_sum.std():.2f})"
    )
    print(
        f"Average time during the extended pilot: {ai_extended_sum.mean():.2f} "
        f"({ai_extended_sum.std():.2f})"
    )


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(10, 10))
baseline_sum.hist(bins=100, ax=ax1)
ai_sum.hist(bins=100, ax=ax2)

ax1.set_ylabel("Frequency")
ax1.set_title("Baseline Distribution")
ax2.set_xlabel("Totaal aantal minuten geschreven per patient")
ax2.set_ylabel("Frequency")
ax2.set_title("AI pilot/ PMS Distribution")

fig.show()

In [ ]:
baseline_sum.plot.density(label="Baseline", color="red")
ai_sum.plot.density(label=NAME, color="blue")
plt.legend()

### Differences in average time between departments

In [ ]:
baseline_average_per_department = baseline_strict_no_short_df.groupby("Afdeling")[
    "Schrijven_minuten"
].mean()
ai_average_per_department = ai_no_short_df.groupby("Afdeling")[
    "Schrijven_minuten"
].mean()

print("Average time per department in baseline:")
print(baseline_average_per_department.apply(lambda x: f"{x:.2f}"))
print("Average time per department in AI pilot:")
print(ai_average_per_department.apply(lambda x: f"{x:.2f}"))

### Results statistical significance

In [ ]:
def get_statistical_significance(variable_1, variable_2):
    # Perform the Mann-Whitney U test
    stat, p = mannwhitneyu(variable_1, variable_2, alternative="two-sided")

    # Output results
    print("Mann-Whitney U statistic:", stat)
    print("p-value:", p)

    # Interpret the results
    if p < 0.05:
        print("There is a statistically significant difference.")
    else:
        print("No statistically significant difference.")

In [ ]:
print(f"Baseline vs AI {NAME}")
get_statistical_significance(baseline_sum, ai_sum)
print("\n")

print(f"Baseline short vs AI {NAME} short (strict)")
get_statistical_significance(baseline_short_sum, ai_short_sum)
print("\n")

print(f"Baseline long vs AI {NAME} long (strict)")
get_statistical_significance(baseline_long_sum, ai_long_sum)
print("\n")


if MODE != MeasureMode.PMS and MODE == MeasureMode.PILOT:
    print("First Half Pilot vs Second Half Pilot")
    get_statistical_significance(ai_first_half_sum, ai_second_half_sum)
    print("\n")

    print("First Half Pilot vs Second Half Pilot (strict)")
    get_statistical_significance(ai_first_half_strict_sum, ai_second_half_strict_sum)
    print("\n")

    print("Second Half Pilot vs Post Pilot (strict)")
    get_statistical_significance(ai_second_half_strict_sum, ai_post_strict_sum)
    print("\n")

    print("Baseline vs second half AI pilot")
    get_statistical_significance(baseline_sum, ai_second_half_sum)
    print("\n")

    print("Baseline vs second half AI pilot (strict)")
    get_statistical_significance(baseline_strict_sum, ai_second_half_strict_sum)
    print("\n")

### Statistical differences per department 

In [ ]:
for department in baseline_average_per_department.index:
    print(f"Department: {department}")
    baseline_department = (
        baseline_strict_no_short_df.loc[
            baseline_strict_no_short_df["Afdeling"] == department
        ]
        .groupby("PatientID")["Schrijven_minuten"]
        .sum()
    )
    ai_department = (
        ai_no_short_df.loc[ai_no_short_df["Afdeling"] == department]
        .groupby("PatientID")["Schrijven_minuten"]
        .sum()
    )
    get_statistical_significance(baseline_department, ai_department)
    print("\n")